<a href="https://colab.research.google.com/github/venkatgh9/IntelligentAgents/blob/main/EA_CrewAI_agent_demo_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to the CrewAI Demo Notebook

Welcome to this interactive demonstration of CrewAI! This notebook illustrates how to orchestrate multiple AI agents to collaborate on a complex task. Here, we will simulate a content creation workflow where agents research a topic, write an article, and prepare social media posts. You'll see how agents can pass information between each other, use specialized tools, and work together to achieve a common goal.

Follow along as we set up our environment, define our agents and their capabilities, configure their tasks, and finally, kick off the crew to generate content from start to finish.

> **Research a topic, generate content and get social media ready post and a blog article!**

### Install/Upgrade Dependencies

This code cell uses `pip install` to download and install all the necessary Python libraries required for this notebook to run. The `-qqqqU` flags ensure a quiet installation (less output) and that packages are upgraded to their latest versions if already installed.

-   `crewai`: The core library for building and managing AI crews.
-   `crewai-tools`: Provides pre-built tools that agents can use (though custom tools are also demonstrated).
-   `langchain`, `langchain-core`, `langchain-openai`: Libraries that provide the framework for integrating Large Language Models (LLMs) and building agent-based applications.
-   `transformers`: Often used for more advanced NLP tasks, though its primary use here might be for underlying dependencies.
-   `requests`: A library for making HTTP requests, used here for potentially sending data to webhooks.

> When using `%pip install` in Colab, a runtime restart is usually not necessary.



In [ ]:
%pip install -qqqqU crewai crewai-tools langchain langchain-core langchain-openai transformers requests

ERROR: Operation cancelled by user


### 🔐 Core Imports & OpenAI API Key Setup

This cell imports essential Python libraries and modules needed throughout the notebook:

-   `os`: For interacting with the operating system, like setting environment variables.
-   `numpy`: A fundamental package for numerical computing in Python (though not directly used in the CrewAI logic, it's a common data science import).
-   `Agent`, `Task`, `Crew` from `crewai`: These are the main building blocks for defining agents, their tasks, and orchestrating them into a crew.
-   `ChatOpenAI` from `langchain_openai`: This class is used to interface with OpenAI's chat models, such as `gpt-4o-mini` or `gpt-4o`, acting as the brain for our agents.
-   `base64`, `requests`: `requests` is used for making web calls (e.g., to a webhook), `base64` is for encoding/decoding data (though not actively used in the final version of the provided code, it's a common utility).
-   `userdata` from `google.colab`: This module is specific to Google Colab and allows secure access to user secrets, such as API keys, stored in Colab's secrets manager.
-   `BaseModel`, `Field` from `pydantic`: Used for data validation and settings management, especially useful when defining structured inputs for tools.
-   `StructuredTool` from `langchain_core.tools`: A class for defining tools with structured inputs, which agents can then use.
-   `Annotated` from `typing`: Used for adding metadata to type hints, often seen with Pydantic fields.

-   `Optional`, `Type` from `typing`: Used for type hinting, making the code more readable and robust by specifying optional parameters and types.

This cell lays the groundwork by importing all necessary components before they are used to define agents, tasks, and tools.

In [ ]:
import base64
import os
import requests
from typing import Annotated, Optional, Type

import numpy as np
import torch
from crewai import Agent, Crew, Task
from google.colab import userdata
from langchain_core.tools import StructuredTool
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

### Setting OpenAI API Key (using Colab Userdata)

This cell demonstrates setting your OpenAI API Key by retrieving it from Google Colab's built-in 'Secrets' manager using `userdata.get('OPENAI_API_KEY')`. This is the recommended and most secure way to handle sensitive credentials in Colab notebooks, as the key is stored outside the notebook content and is not exposed in code or output. Once retrieved, the key is stored in an environment variable named `OPENAI_API_KEY`, making it accessible to other parts of the program, such as the ChatOpenAI model.

In [ ]:
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

### Initializing the Language Model (LLM)

This cell initializes the `ChatOpenAI` object, which serves as the underlying Large Language Model (LLM) for our agents.

-   `model="gpt-4o"`: Specifies that the `gpt-4o` model will be used. This is a powerful, multimodal model from OpenAI.
-   `openai_api_key=os.environ["OPENAI_API_KEY"]`: Passes your retrieved OpenAI API key to the LLM. This allows the `ChatOpenAI` instance to authenticate and make requests to OpenAI's services.

This `llm` object will then be assigned to the agents, enabling them to understand instructions and generate responses.

In [ ]:
llm = ChatOpenAI(model="gpt-4o",
                 openai_api_key=os.environ["OPENAI_API_KEY"])

### Importing `BaseTool`

This cell specifically imports `BaseTool` from `crewai.tools`.  

`BaseTool` is a foundational class in CrewAI (or `crewai-tools`) that you extend when creating custom tools for your agents. By inheriting from `BaseTool`, your custom tool classes gain the necessary structure and methods to be recognized and utilized by CrewAI agents.

In [ ]:
from crewai.tools import BaseTool

### Concrete Tool Implementations

This cell defines the actual logic for two custom tools and then wraps them into CrewAI-compatible `BaseTool` classes:

1.  **`_make_social_copy_impl(title, summary, platform)`**: This is a helper function that uses a `ChatOpenAI` model (`gpt-4o-mini`) to generate a concise social media post. It takes a title, summary, and target platform (e.g., 'linkedin', 'twitter') and crafts a post following specific constraints (punchy lines, CTA, hashtags).

2.  **`_post_via_webhook_impl(text, image_path)`**: This helper function simulates publishing a post by sending its `text` and an optional `image_path` to a predefined `SOCIAL_WEBHOOK_URL` via an HTTP POST request. It includes error handling for the request.

3.  **`MakeSocialCopyTool` (Class)**: This class inherits from `BaseTool` and wraps `_make_social_copy_impl`. It defines:
    -   `name`: A unique identifier for the tool.
    -   `description`: Explains what the tool does and its arguments, which helps agents understand when and how to use it.
    -   `_run`: The method that CrewAI agents call to execute the tool's logic.

4.  **`PostViaWebhookTool` (Class)**: Similar to `MakeSocialCopyTool`, this class wraps `_post_via_webhook_impl`, providing a `name`, `description`, and `_run` method for the webhook functionality.

Finally, instances of these tool classes (`make_social_copy`, `post_via_webhook`) are created, which can then be assigned to agents.

In [ ]:
def _make_social_copy_impl(title: str, summary: str, platform: str = "linkedin") -> str:
    """
    Generates a concise, platform-optimized social media post.

    Args:
        title (str): The title of the article or content.
        summary (str): A summary of the article or content.
        platform (str, optional): The target social media platform (e.g., "linkedin", "twitter").
                                  Defaults to "linkedin".

    Returns:
        str: The generated social media post.
    """
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt = (
        f"Craft a concise {platform} post promoting an article.\n"
        f"Title: {title}\nSummary: {summary}\n"
        "Constraints: 1-2 punchy lines, clear CTA, 3-5 relevant hashtags."
    )
    return llm.invoke(prompt).content

def _post_via_webhook_impl(text: str, image_path: Optional[str] = None) -> str:
    """
    Publishes a post (and optional image) via a webhook integration.

    This function sends the provided text and an optional image to a predefined
    webhook URL. It's designed to simulate publishing content to an external service.

    Args:
        text (str): The text content of the post.
        image_path (Optional[str], optional): The file path to an image to be included.
                                               Defaults to None.

    Returns:
        str: A confirmation message, typically "Posted" if successful.

    Raises:
        requests.exceptions.RequestException: If the HTTP request to the webhook fails.
    """
    WEBHOOK_URL = os.environ["SOCIAL_WEBHOOK_URL"]
    files = {"image": open(image_path, "rb")} if image_path else None
    r = requests.post(WEBHOOK_URL, data={"text": text}, files=files, timeout=60)
    r.raise_for_status()
    return "Posted"

class MakeSocialCopyTool(BaseTool):
    """A tool to create a short, platform-optimized social media post."""
    name: str = "make_social_copy"
    description: str = "Create a short, platform-optimized social post from a title and summary. Args: title (str), summary (str), platform (str, optional)"

    def _run(self, title: str, summary: str, platform: str = "linkedin") -> str:
        return _make_social_copy_impl(title, summary, platform)

class PostViaWebhookTool(BaseTool):
    """A tool to publish a post and optional image via a webhook integration."""
    name: str = "post_via_webhook"
    description: str = "Publish a post (and optional image) via a webhook integration. Args: text (str), image_path (str, optional)"

    def _run(self, text: str, image_path: Optional[str] = None) -> str:
        return _post_via_webhook_impl(text, image_path)

# Create tool instances
make_social_copy = MakeSocialCopyTool()
post_via_webhook = PostViaWebhookTool()

## 🔁 Function-Based Tool Fallbacks
**Ensuring Compatibility & Execution Stability**

In CrewAI, **Tools** are the specific skills you give your agents (e.g., searching the web, reading a file). However, because the library is evolving rapidly, there are two primary ways to define these skills. This section provides a "Plan B" using standard Python functions.

### Why use Function-Based Fallbacks?
While Class-based tools are excellent for complex logic, they can sometimes cause "Parsing Errors" or compatibility issues with certain LLM models or older versions of the CrewAI framework.

| Feature | **Class-Based Tools** | **Function-Based Tools** |
| :--- | :--- | :--- |
| **Structure** | Uses Python Classes (`BaseTool`) | Uses standard functions (`def`) |
| **Complexity** | Best for tools requiring state/memory | Best for direct, stateless actions |
| **Compatibility** | Subject to library updates | High (Universal standard) |
| **Reliability** | More "boilerplate" code | Minimalist; easier for the LLM to parse |

### The "Safety Net" Strategy
By providing these equivalents, we ensure the notebook is **future-proof**:
1. **Redundancy:** If the class-based tool fails to initialize, the agent can use the function.
2. **Precision:** Simple functions often reduce "hallucinations" because the instructions are more direct.
3. **Versatility:** Keeps the code compatible across different versions of `crewai` and `langchain`.


In [ ]:
def create_function_tools():
    """
    Creates and returns a set of function-based tools that CrewAI can accept.
    This serves as an alternative to class-based tools, particularly for compatibility.
    """

    def make_social_copy_func(title: str, summary: str, platform: str = "linkedin") -> str:
        """
        Create a short, platform-optimized social post from a title and summary.

        Args:
            title (str): The title of the article or content.
            summary (str): A summary of the article or content.
            platform (str, optional): The target social media platform.

        Returns:
            str: The generated social media post.
        """
        return _make_social_copy_impl(title, summary, platform)

    def post_via_webhook_func(text: str, image_path: Optional[str] = None) -> str:
        """
        Publish a post (and optional image) via a webhook integration.

        Args:
            text (str): The text content of the post.
            image_path (Optional[str], optional): The file path to an image to be included.

        Returns:
            str: A confirmation message, typically "Posted" if successful.
        """
        return _post_via_webhook_impl(text, image_path)

    # return generate_image_local_func, make_social_copy_func, post_via_webhook_func
    return make_social_copy_func, post_via_webhook_func

# Create function-based tools as backup
make_social_func, post_webhook_func = create_function_tools()

### Defining the Agents

This cell defines the three core agents that will participate in our content creation workflow:

1.  **`llm = ChatOpenAI(...)`**: The `ChatOpenAI` instance initialized earlier (`gpt-4o-mini`) is assigned as the Large Language Model (LLM) that powers each agent. This means all agents will use this specific OpenAI model for their reasoning and text generation.

2.  **`researcher` Agent**:
    -   `role`: "Researcher" - Clearly defines its primary job.
    -   `goal`: "Collect 5 up-to-date, credible insights on the topic." - What the agent aims to achieve.
    -   `backstory`: "Finds and synthesizes sources." - Provides context for its behavior.
    -   `allow_delegation=False`: This agent will not delegate its tasks to other agents.

3.  **`writer` Agent**:
    -   `role`: "Writer" - Its job is to write.
    -   `goal`: "Draft a 600-900 word article from notes with clear structure." - Its main objective.
    -   `backstory`: "Senior technical writer." - Influences its writing style and quality.
    -   `allow_delegation=False`: This agent also works independently on its assigned tasks.

4.  **`social_agent` Agent**:
    -   `role`: "Social Media Strategist" - Focused on social media.
    -   `goal`: "Adapt article into a platform-optimized post and (optionally) publish." - Its target outcome.
    -   `backstory`: "Knows tone/length/hashtag best practices." - Specifies its expertise.
    -   `tools=[make_social_copy, post_via_webhook]`: Crucially, this agent is equipped with the custom `MakeSocialCopyTool` and `PostViaWebhookTool` defined earlier, allowing it to generate social media content and simulate publishing it.
    -   `allow_delegation=False`: This agent also performs its tasks directly.

Each agent is given a specific role, goal, and backstory to guide its behavior, along with the `llm` and, for the `social_agent`, specific tools to execute its tasks.

In [ ]:
from crewai import Agent

# Make sure to set your OpenAI API key
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

researcher = Agent(
    role="Researcher",
    goal="Collect 5 up-to-date, credible insights on the topic.",
    backstory="Finds and synthesizes sources.",
    allow_delegation=False,
    llm=llm,
)

writer = Agent(
    role="Writer",
    goal="Draft a 600-900 word article from notes with clear structure.",
    backstory="Senior technical writer.",
    allow_delegation=False,
    llm=llm,
)

# Try using the class-based tools first, fallback to functions if needed
try:
  social_agent = Agent(
      role="Social Media Strategist",
      goal="Adapt article into a platform-optimized post and (optionally) publish.",
      backstory="Knows tone/length/hashtag best practices.",
      tools=[make_social_copy, post_via_webhook],
      allow_delegation=False,
      llm=llm,
  )
  print("Successfully created agents with class-based tools")

except Exception as e:
    print(f"Class-based tools failed: {e}")
    print("Trying function-based tools...")

    social_agent = Agent(
        role="Social Media Strategist",
        goal="Adapt article into a platform-optimized post and (optionally) publish.",
        backstory="Knows tone/length/hashtag best practices.",
        tools=[make_social_func, post_webhook_func],
        allow_delegation=False,
        llm=llm,
    )
    print("Successfully created agents with function-based tools")

print("All agents created successfully!")

Successfully created agents with class-based tools
All agents created successfully!


### Defining the Tasks

This cell defines the sequence of tasks that the agents will execute, showcasing how outputs from one task can become context for another:

1.  **`t1` (Research Task)**:
    -   `description`: Specifies what the researcher needs to do: find 5 key insights on 'State of Canada's economic growth in the 21st century' with current developments, challenges, opportunities, and references.
    -   `agent`: Assigned to the `researcher` agent.
    -   `expected_output`: A clear instruction on the format of the output (bulleted list with citations).

2.  **`t2` (Writing Task)**:
    -   `description`: Instructs the writer to create a structured article (250-300 words) from the research notes, including a title, introduction, sections, and conclusion.
    -   `agent`: Assigned to the `writer` agent.
    -   `expected_output`: A complete markdown article.
    -   `context=[t1]`: This is vital! It means the output of `t1` (the research insights) will be provided as input context to `t2`, allowing the writer to base the article on the research.

3.  **`t3` (Social Media Task)**:
    -   `description`: Outlines the social media strategist's job: create LinkedIn and X/Twitter posts using the `make_social_copy` tool, extracting title and summary from the article, and explicitly states *not* to publish them but to show drafts.
    -   `agent`: Assigned to the `social_agent` agent.
    -   `expected_output`: The drafted LinkedIn and X/Twitter posts, along with a publishing status.
    -   `context=[t1, t2]`: This task receives the output from both the research (`t1`) and writing (`t2`) tasks. The `social_agent` will primarily use the `t2` output (the article) to extract the title and summary for its social media posts.

In [ ]:
t1 = Task(
    description="Research the topic: 'State of Canada's economic growth in the 21st century'. Find 5 key insights with current developments, challenges, and opportunities. Include brief references to sources.",
    agent=researcher,
    expected_output="Bulleted list of 5 insights with short citations or source references.",
)

t2 = Task(
    description="Write a structured 250 to 300 word article from the research notes. Include: compelling title, introduction, 3-4 main sections, and conclusion. Start with a brief summary paragraph.",
    agent=writer,
    expected_output="Complete markdown article with title, summary paragraph, and well-structured sections.",
    context=[t1],
)


t3 = Task(
    description="""Create social media posts for the article:
    1. Use make_social_copy to create a LinkedIn version (title from article, summary from article, platform='linkedin')
    2. Use make_social_copy to create an X/Twitter version (title from article, summary from article, platform='twitter')
    3. For testing purposes, do NOT publish automatically - just show what the posts would look like

    Extract the title and summary from the article context to use with the tools.""",
    agent=social_agent,
    expected_output="LinkedIn post version, X/Twitter post version, and publishing status.",
    context=[t1, t2],
)

### Building and Running the Crew

This cell orchestrates the entire workflow:

1.  **`crew = Crew(...)`**: A `Crew` instance is created, which brings together:
    -   `agents=[researcher, writer, social_agent]`: All the defined agents are included in the crew.
    -   `tasks=[t1, t2, t3]`: The sequence of tasks to be executed is provided.
    -   `verbose=True`: This setting ensures that detailed logs and progress updates from the agents and tasks are printed to the console during execution, allowing you to observe their thought process and actions.

2.  **`print("🚀 Starting CrewAI workflow...")`**: A simple print statement to indicate the start of the process.

3.  **`try...except` block**:
    -   `result = crew.kickoff()`: This is the command that initiates the entire CrewAI workflow. The agents will start processing their tasks in the defined order, passing context between them. The final output of the last task will be stored in the `result` variable.
    -   Error Handling: The `try...except` block is included to catch any exceptions that might occur during the workflow (e.g., API key issues, network errors, unexpected agent behavior). If an error occurs, it will print a descriptive message, helping you troubleshoot issues.

After successful completion, the final `result` (the social media posts generated by the `social_agent`) is printed, showing the culmination of the agents' collaborative efforts.

In [ ]:
crew = Crew(
    agents=[researcher, writer, social_agent],
    tasks=[t1, t2, t3],
    verbose=True  # Set to True or 2 for detailed output
)

print("🚀 Starting CrewAI workflow...")
print("=" * 50)

try:
    result = crew.kickoff()
    print("\n" + "=" * 50)
    print("✅ WORKFLOW COMPLETED!")
    print("=" * 50)
    print(result)
except Exception as e:
    print(f"❌ Error running crew: {e}")
    print("Check your API keys and environment setup.")

🚀 Starting CrewAI workflow...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6ad5f729-4050-4f6c-a36a-02f7af6110c7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Task: Research the topic: 'State of Canada's economic growth in the 21st century'. Find 5 key insights with    │
│  current developments, challenges, and opportunities. Include brief references to sources.                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Social Media Strategist                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **LinkedIn Post Version:**                                                                                     │
│  "🇨🇦 Canada’s Economic Renaissance: Navigating Post-Pandemic Challenges and Opportunities 🌟                    │
│                                                                                                                 │
│  As Canada emerges from the COVID-19 pandemic, it showcases a promising economic recovery driven by strong      │
│  consumer spending and trade partnerships. However, challenges such as inflation and labor market skill gaps    │
│  persist, requiring strategic policymaking. #CanadaEconomy #PostPandemic #TradePartnerships #Sustainability"    │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **X/Twitter Post Version:**                                                                                    │
│  "🇨🇦 Canada’s Economic Renaissance: Navigating Post-Pandemic Challenges 🌟                                      │
│                                                                                                                 │
│  Canada is witnessing a strong economic recovery post-COVID-19. However, inflation and labor market skill gaps  │
│  remain critical challenges. Policymakers must take a balanced approach to sustain growth. #CanadaEconomy       │
│  #PostPandemic"                                                                                                 │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Publishing Status:** Not published automatically for testing purposes.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 71e3173e-ab89-4019-a704-9bb55828d247                                                                     │
│  Agent: Social Media Strategist                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✅ WORKFLOW COMPLETED!
**LinkedIn Post Version:**
"🇨🇦 Canada’s Economic Renaissance: Navigating Post-Pandemic Challenges and Opportunities 🌟 

As Canada emerges from the COVID-19 pandemic, it showcases a promising economic recovery driven by strong consumer spending and trade partnerships. However, challenges such as inflation and labor market skill gaps persist, requiring strategic policymaking. #CanadaEconomy #PostPandemic #TradePartnerships #Sustainability"

---

**X/Twitter Post Version:**
"🇨🇦 Canada’s Economic Renaissance: Navigating Post-Pandemic Challenges 🌟 

Canada is witnessing a strong economic recovery post-COVID-19. However, inflation and labor market skill gaps remain critical challenges. Policymakers must take a balanced approach to sustain growth. #CanadaEconomy #PostPandemic"

---

**Publishing Status:** Not published automatically for testing purposes.


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6ad5f729-4050-4f6c-a36a-02f7af6110c7                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **LinkedIn Post Version:**                                                                       │
│  "🇨🇦 Canada’s Economic Renaissance: Navigating Post-Pandemic Challenges and Opportunities 🌟                    │
│                                                                                                                 │
│  As Canada emerges from the COVID-19 pandemic, it showcases a promising economic recovery driven by strong      │
│  consumer spending and trade partnerships. However, challenges such as inflation and labor market skill gaps    │
│  persist, requiring strategic policymaking. #CanadaEconomy #PostPandemic #TradePartnerships #Sustainability"    │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **X/Twitter Post Version:**                                                                                    │
│  "🇨🇦 Canada’s Economic Renaissance: Navigating Post-Pandemic Challenges 🌟                                      │
│                                                                                                                 │
│  Canada is witnessing a strong economic recovery post-COVID-19. However, inflation and labor market skill gaps  │
│  remain critical challenges. Policymakers must take a balanced approach to sustain growth. #CanadaEconomy       │
│  #PostPandemic"                                                                                                 │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Publishing Status:** Not published automatically for testing purposes.                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Conclusion

Congratulations! You've successfully run a multi-agent workflow using CrewAI. This notebook demonstrated how to:

-   **Define specialized AI agents** with unique roles, goals, and backstories.
-   **Create custom tools** that agents can use to interact with external systems or perform specific actions.
-   **Orchestrate a sequence of tasks**, where agents collaborate and pass information (context) from one stage to the next.
-   **Execute a complete workflow** from research to content generation and social media preparation.

This example provides a foundation for building more complex and sophisticated agentic systems. You can expand on this by adding more agents, integrating different types of tools (e.g., web search, image generation, data analysis), and defining more intricate task dependencies to automate various processes.